In [1]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

train  = pd.read_csv("aug_train.csv")
train.head()

test  = pd.read_csv("aug_test.csv")
test.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_type,last_new_job,training_hours,target
0,0.624,NaN,Has relevent experience,Full time course,Graduate,Other,3,Pvt Ltd,1,134,0
1,0.920,Female,No relevent experience,no_enrollment,Graduate,STEM,5,Early Stage Startup,1,34,1
2,0.767,NaN,Has relevent experience,Full time course,Graduate,STEM,10,Pvt Ltd,2,90,0
3,0.910,Male,No relevent experience,NaN,High School,NaN,10,NaN,never,42,0
4,0.624,Male,Has relevent experience,Part time course,Graduate,STEM,3,Pvt Ltd,1,198,0


In [2]:
#Task1 Data clean, imputation

#1. in experience, replace >20 to 21; <1 to 1, and convert this as a numerical column
train.loc[train["experience"] == ">20", "experience"] = 21
train.loc[train["experience"] == "<1", "experience"] = 1
train.tail()

test.loc[test["experience"] == ">20", "experience"] = 21
test.loc[test["experience"] == "<1", "experience"] = 1

test["experience"] = pd.to_numeric(test["experience"])
train["experience"] = pd.to_numeric(train["experience"])

#train["experience"].describe()
#test["experience"].describe()

#2. in last_new_job, replace >4 to 5; never to 0, and convert this as a numerical column
train.loc[train["last_new_job"] == ">4", "last_new_job"] = 5
train.loc[train["last_new_job"] == "never", "last_new_job"] = 0
train["last_new_job"] = pd.to_numeric(train["last_new_job"])
#train["last_new_job"].describe()

test.loc[test["last_new_job"] == ">4", "last_new_job"] = 5
test.loc[test["last_new_job"] == "never", "last_new_job"] = 0
test["last_new_job"] = pd.to_numeric(test["last_new_job"])
#test["last_new_job"].describe()



In [3]:
#3. If the column is categorical, impute the missing value as its mode. 
# #If the column is numerical, impute the missing value as its median

#Convert categorical data into numeric data

for column in test:
    if test[column].dtype == "object":       
        test[column] = test[column].fillna(test[column].mode()[0])
    else:
        test[column] = test[column].fillna(test[column].median())

for column in train:
    if train[column].dtype == "object":       
        train[column] = train[column].fillna(train[column].mode()[0])
    else:
        train[column] = train[column].fillna(train[column].median())

test.head()




train.head()


,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_type,last_new_job,training_hours,target
0,0.624,Male,No relevent experience,no_enrollment,High School,STEM,5.0,Pvt Ltd,0.0,21,0
1,0.926,Male,Has relevent experience,no_enrollment,Graduate,STEM,21.0,Pvt Ltd,5.0,12,0
2,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,21.0,Public Sector,5.0,26,0
3,0.624,Male,No relevent experience,Full time course,High School,STEM,1.0,Pvt Ltd,0.0,30,1
4,0.920,Female,Has relevent experience,no_enrollment,Masters,STEM,21.0,Pvt Ltd,5.0,46,0


In [4]:


#get X and y
y_train = train[["target"]]
X_train = train[["city_development_index","gender","relevent_experience","enrolled_university","education_level","major_discipline","experience","company_type","last_new_job","training_hours"]]
y_test = test[["target"]]
X_test = test[["city_development_index","gender","relevent_experience","enrolled_university","education_level","major_discipline","experience","company_type","last_new_job","training_hours"]]


#convert categorical to numeric
categorical_columns =['gender','relevent_experience', 'enrolled_university', 'education_level','major_discipline','company_type']
X_train = pd.get_dummies(X_train, columns=categorical_columns )
X_test = pd.get_dummies(X_test, columns=categorical_columns )

X_test.head()

,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,gender_Other,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,enrolled_university_Full time course,...,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_Other,major_discipline_STEM,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Public Sector,company_type_Pvt Ltd
0,0.624,3,1,134,0,1,0,1,0,1,...,0,0,0,1,0,0,0,0,0,1
1,0.920,5,1,34,1,0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,0
2,0.767,10,2,90,0,1,0,1,0,1,...,0,0,0,0,1,0,0,0,0,1
3,0.910,10,0,42,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
4,0.624,3,1,198,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1


In [6]:
#Task2 Classification
#1. Build a classification model from the training set ( you can use any algorithm)
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)

print(prediction)
#2. generate the confusion matrix for the training set and calculate the accuracy, precision, recall, and F1-score
#3. Applying the model in the test set and generating the prediction
#4. generate the confusion matrix from the test set and calculate the accuracy, precision, recall, and F1-score
#5. compare the results between the training and test set
#from sklearn import svm
#clf = svm.SVC()
#clf.fit(X_train, y_train)
#SCV()

ValueError: Found input variables with inconsistent numbers of samples: [60900, 2100]